In [1]:
import random
import numpy as np
from collections import deque

from deap import base
from deap import creator
from deap import tools

In [2]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

IND_SIZE = 8 * 14

INT_MIN, INT_MAX = 0, 9
toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, INT_MIN, INT_MAX)
toolbox.register("individual", tools.initRepeat, creator.Individual, 
                 toolbox.attr_int, IND_SIZE)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [3]:
def eval_814_num(individual):
    # Reshape individual to 8x14 grid using numpy
    grid = np.array(individual).reshape(8, 14)
    
    # Define 8 possible directions
    deltas = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    
    def is_valid(r, c):
        return 0 <= r < 8 and 0 <= c < 14
    
    def can_form(n):
        S = str(n)
        l = len(S)
        dig = [int(d) for d in S]
        
        q = deque()
        visited = set()
        has_start = False
        
        for r in range(8):
            for c in range(14):
                if grid[r, c] == dig[0]:
                    has_start = True
                    state = (1, r, c)
                    if state not in visited:
                        visited.add(state)
                        q.append((r, c, 1))
        
        if l == 1:
            return has_start
        
        while q:
            cr, cc, cpos = q.popleft()
            if cpos == l:
                return True
            for dr, dc in deltas:
                nr = cr + dr
                nc = cc + dc
                if is_valid(nr, nc) and grid[nr, nc] == dig[cpos]:
                    new_state = (cpos + 1, nr, nc)
                    if new_state not in visited:
                        visited.add(new_state)
                        q.append((nr, nc, cpos + 1))
        return False
    
    # Find the largest X
    X = 0
    n = 1
    while n < 20000: 
        if not can_form(n):
            return (X,)
        X = n
        n += 1
    return (X,)

In [4]:
toolbox.register("evaluate", eval_814_num)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=9, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [5]:
def main():
    pop = toolbox.population(n=300)
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    CXPB, MUTPB = 0.5, 0.2
    fits = [ind.fitness.values[0] for ind in pop]
    g = 0

    while max(fits) < 5000 and g < 300:
        g = g + 1
    
         # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        if g % 10 == 0:
            print("-- Generation %i --" % g)
            length = len(pop)
            mean = sum(fits) / length
            sum2 = sum(x*x for x in fits)
            std = abs(sum2 / length - mean**2)**0.5

            print("  Min %s" % min(fits))
            print("  Max %s" % max(fits))
            print("  Avg %s" % mean)
            print("  Std %s" % std)

    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s %s" % (best_ind, best_ind.fitness.values))
    for i in range(8):
        row = ''.join(map(str, best_ind[i*14:(i+1)*14]))
        print(row)

In [6]:
if __name__ == "__main__":
    main()

-- Generation 10 --
  Min 21.0
  Max 448.0
  Avg 196.18333333333334
  Std 85.22266749847459
-- Generation 20 --
  Min 21.0
  Max 507.0
  Avg 260.44666666666666
  Std 130.0949671927738
-- Generation 30 --
  Min 21.0
  Max 507.0
  Avg 433.43666666666667
  Std 128.62153521950447
-- Generation 40 --
  Min 76.0
  Max 586.0
  Avg 476.8233333333333
  Std 144.4779756764178
-- Generation 50 --
  Min 66.0
  Max 769.0
  Avg 519.7733333333333
  Std 176.04850266017283
-- Generation 60 --
  Min 65.0
  Max 828.0
  Avg 653.18
  Std 229.34992391540067
-- Generation 70 --
  Min 21.0
  Max 1045.0
  Avg 810.86
  Std 281.5095032143675
-- Generation 80 --
  Min 21.0
  Max 1045.0
  Avg 863.15
  Std 342.59510723301344
-- Generation 90 --
  Min 21.0
  Max 1055.0
  Avg 858.49
  Std 342.39711335426483
-- Generation 100 --
  Min 21.0
  Max 1105.0
  Avg 923.95
  Std 338.23382567882436
-- Generation 110 --
  Min 21.0
  Max 1161.0
  Avg 947.11
  Std 349.98830156639997
-- Generation 120 --
  Min 43.0
  Max 1175.0
  A